<a href="https://colab.research.google.com/github/minzee09/food-nutrition-image-recognizer-AI/blob/main/food_nutrition_recognition_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

# tf.data 함수에 대해 디버그 모드 활성화
tf.data.experimental.enable_debug_mode()

from google.colab import drive
drive.mount('/content/drive')

import os
save_dir = '/content/drive/MyDrive'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
checkpoint_path = os.path.join(save_dir, 'best_food_model.h5')


Mounted at /content/drive


In [ ]:
# TPU 설정
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

# tf.config.experimental_connect_to_cluster(resolver)
# tf.tpu.experimental.initialize_tpu_system(resolver)
# strategy = tf.distribute.TPUStrategy(resolver)


KeyError: 'COLAB_TPU_ADDR'

In [ ]:
# 데이터셋 로드
food101_dataset, info = tfds.load('food101', split='train', with_info=True, as_supervised=True)

# 카테고리 출력
categories = info.features['label'].names
print("Categories in Food101 dataset:", categories)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/75750 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/food101/incomplete.R9324W_2.0.0/food101-train.tfrecord*...:   0%|         …

Generating validation examples...:   0%|          | 0/25250 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/food101/incomplete.R9324W_2.0.0/food101-validation.tfrecord*...:   0%|    …

Dataset food101 downloaded and prepared to /root/tensorflow_datasets/food101/2.0.0. Subsequent calls will reuse this data.
Categories in Food101 dataset: ['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito', 'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake', 'ceviche', 'cheesecake', 'cheese_plate', 'chicken_curry', 'chicken_quesadilla', 'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder', 'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes', 'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict', 'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras', 'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari', 'fried_rice', 'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad', 'grilled_cheese_sandwich', 'grilled_salmon', 'guacamole', 'gyoza', 'hamburger', 'hot_and_sour_soup'

In [ ]:
# 카테고리를 숫자로 인코딩
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
categories_num = le.fit_transform(categories)

# 각 음식이름을 해당하는 숫자 ID로 매핑
name_to_id = {name: id for id, name in enumerate(le.classes_)}


In [ ]:
# 영양 정보 매핑
nutrition_mapping = {
    'apple_pie': {'sugar': 12.5, 'carbs': 42.5, 'protein': 6.25, 'fat': 16.25},
    'baby_back_ribs': {'sugar': 7.14, 'carbs': 10.71, 'protein': 14.29, 'fat': 14.29},
    'baklava': {'sugar': 12.8, 'carbs': 37.8, 'protein': 6.6, 'fat': 29},
    'beef_carpaccio': {'sugar': 0.7, 'carbs': 2, 'protein': 13, 'fat': 13},
    'beef_tartare': {'sugar': 0.08, 'carbs': 0.34, 'protein': 17.4, 'fat': 15.6},
    'beet_salad': {'sugar': 7, 'carbs': 11, 'protein': 2, 'fat': 0.4},
    'beignets': {'sugar': 31.1, 'carbs': 53.7, 'protein': 5.46, 'fat': 20.2},
    'bibimbap': {'sugar': 0.05, 'carbs': 18.84, 'protein': 6.86, 'fat': 4.32},
    'bread_pudding': {'sugar': 15.1, 'carbs': 23.5, 'protein': 5.6, 'fat': 4.86},
    'breakfast_burrito': {'sugar': 2.11, 'carbs': 21.58, 'protein': 9.47, 'fat': 8.95},
    'bruschetta': {'sugar': 3.34, 'carbs': 21.3, 'protein': 4.29, 'fat': 7.61},
    'caesar_salad': {'sugar': 1.6, 'carbs': 8.4, 'protein': 8.4, 'fat': 5.6},
    'cannoli': {'sugar': 13.1, 'carbs': 35.1, 'protein': 7.52, 'fat': 10.9},
    'caprese_salad': {'sugar': 6.83, 'carbs': 9.97, 'protein': 4.09, 'fat': 2.15},
    'carrot_cake': {'sugar': 22.63, 'carbs': 25.26, 'protein': 3.68, 'fat': 16.84},
    'ceviche': {'sugar': 1.34, 'carbs': 3.58, 'protein': 10.5, 'fat': 0.76},
    'cheese_plate': {'sugar': 0.5, 'carbs': 8.3, 'protein': 45, 'fat': 50},
    'cheesecake': {'sugar': 15.28, 'carbs': 28.64, 'protein': 7.23, 'fat': 20.63},
    'chicken_curry': {'sugar': 1.58, 'carbs': 15.1, 'protein': 4.95, 'fat': 3.98},
    'chicken_quesadilla': {'sugar': 2.14, 'carbs': 14, 'protein': 12.86, 'fat':5.86 },
    'chicken_wings': {'sugar': 5.94, 'carbs': 7.28, 'protein': 19, 'fat': 15.5},
    'chocolate_cake': {'sugar': 23.31, 'carbs': 37.33, 'protein': 5.2, 'fat': 25.28},
    'chocolate_mousse': {'sugar': 30, 'carbs': 32, 'protein': 8.4, 'fat': 32},
    'churros': {'sugar': 22.73, 'carbs': 58.17, 'protein': 6.6, 'fat': 11.29},
    'clam_chowder': {'sugar': 2.66, 'carbs': 7.54, 'protein': 3.25, 'fat': 2.1},
    'club_sandwich': {'sugar': 3.03, 'carbs': 18.79, 'protein': 9.7, 'fat': 9.7},
    'crab_cakes': {'sugar': 0.66, 'carbs': 4.48, 'protein': 14.4, 'fat': 15},
    'creme_brulee': {'sugar': 17.9, 'carbs': 17.8, 'protein': 4.06, 'fat': 2.83},
    'croque_madame': {'sugar': 10, 'carbs': 57, 'protein': 37, 'fat': 38},
    'cup_cakes': {'sugar': 25.58, 'carbs': 35.92, 'protein': 5.48, 'fat': 25.2},
    'deviled_eggs': {'sugar': 1.07, 'carbs': 1.07, 'protein': 11.5, 'fat': 15.9},
    'donuts': {'sugar': 33.33, 'carbs': 73.02, 'protein': 3.17, 'fat': 20.63},
    'dumplings': {'sugar': 0.49, 'carbs': 27.79, 'protein': 9.38, 'fat': 2.41},
    'edamame': {'sugar': 2.12, 'carbs': 8.63, 'protein': 11.5, 'fat': 7.58},
    'eggs_benedict': {'sugar': 1.17, 'carbs': 8.48, 'protein': 13.8, 'fat': 22.1},
    'escargots': {'sugar': 0, 'carbs': 2.48, 'protein': 20, 'fat': 6.91},
    'falafel': {'sugar': 4.52, 'carbs': 29, 'protein': 8.28, 'fat': 41.2},
    'filet_mignon': {'sugar': 0, 'carbs': 0, 'protein': 30.5, 'fat': 8.25},
    'fish_and_chips': {'sugar': 2, 'carbs': 71, 'protein': 43, 'fat': 26},
    'foie_gras': {'sugar': 1, 'carbs': 4.7, 'protein': 11, 'fat': 44},
    'french_fries': {'sugar': 0.19, 'carbs': 44.16, 'protein': 3.54, 'fat': 11.3},
    'french_onion_soup': {'sugar': 2.58, 'carbs': 16.2, 'protein': 7.16, 'fat': 6.78},
    'french_toast': {'sugar': 11, 'carbs': 42, 'protein': 6, 'fat': 15},
    'fried_calamari': {'sugar': 0.2, 'carbs': 14.1, 'protein': 13.8, 'fat': 13.2},
    'fried_rice': {'sugar': 1.79, 'carbs': 22.5, 'protein': 6.92, 'fat': 8.38},
    'frozen_yogurt': {'sugar':22.61 , 'carbs': 26.96, 'protein': 4.35, 'fat': 5.22},
    'garlic_bread': {'sugar': 16, 'carbs': 44, 'protein': 6, 'fat': 13.33},
    'gnocchi': {'sugar': 0.14, 'carbs': 9.38, 'protein': 8.51, 'fat': 12},
    'greek_salad': {'sugar': 0.93, 'carbs': 1.49, 'protein': 0.72, 'fat': 3.46},
    'grilled_cheese_sandwich': {'sugar': 5.06, 'carbs': 28.7, 'protein': 10.8, 'fat': 19.3},
    'grilled_salmon': {'sugar': 0.4, 'carbs': 8, 'protein': 20, 'fat': 11},
    'guacamole': {'sugar': 0.72, 'carbs': 8.45, 'protein': 1.95, 'fat': 14.2},
    'gyoza': {'sugar': 2.77, 'carbs': 25.57, 'protein': 7.03, 'fat': 11.5},
    'hamburger': {'sugar': 4.78, 'carbs': 18.01, 'protein': 9.55, 'fat': 13.23},
    'hot_and_sour_soup': {'sugar': 0.42, 'carbs': 4.35, 'protein': 2.58, 'fat': 1.21},
    'hot_dog': {'sugar': 10, 'carbs': 33.75, 'protein': 7.5, 'fat': 11.25},
    'huevos_rancheros': {'sugar': 2.26, 'carbs': 8.85, 'protein': 6.68, 'fat': 6.65},
    'hummus': {'sugar': 2, 'carbs': 33.33, 'protein': 10, 'fat': 24.17},
    'ice_cream': {'sugar': 17.3, 'carbs': 22.25, 'protein': 4.98, 'fat': 7.41},
    'lasagna': {'sugar': 3.33, 'carbs': 16.2, 'protein': 7.45, 'fat': 4.97},
    'lobster_bisque': {'sugar': 1.03, 'carbs': 1.69, 'protein': 5.23, 'fat': 2.61},
    'lobster_roll_sandwich': {'sugar': 4.17, 'carbs': 21.88, 'protein': 5.73, 'fat': 10.94},
    'macaroni_and_cheese': {'sugar': 8.5, 'carbs': 47, 'protein': 9.7, 'fat': 16},
    'macarons': {'sugar': 2.44, 'carbs': 23.1, 'protein': 8.68, 'fat': 10.5},
    'miso_soup': {'sugar': 0.64, 'carbs': 1.54, 'protein': 1.69, 'fat': 0.84},
    'mussels': {'sugar': 0, 'carbs': 4.67, 'protein': 15.1, 'fat': 2.84},
    'nachos': {'sugar': 2, 'carbs': 62, 'protein': 6, 'fat': 29},
    'omelette': {'sugar': 2.7, 'carbs': 20.27, 'protein': 5.86, 'fat': 5.86},
    'onion_rings': {'sugar': 7.33, 'carbs': 70, 'protein': 3.33, 'fat': 20},
    'oysters': {'sugar': 2.62, 'carbs': 18.4, 'protein': 4.96, 'fat': 10.8},
    'pad_thai': {'sugar': 10.5, 'carbs': 25.4, 'protein': 6.98, 'fat': 14.85},
    'paella': {'sugar': 0.67, 'carbs': 15.8, 'protein': 9.54, 'fat': 7.17},
    'pancakes': {'sugar': 7.05, 'carbs': 35.3, 'protein': 7.41, 'fat': 12.1},
    'panna_cotta': {'sugar': 21.11, 'carbs': 31.11, 'protein': 1.11, 'fat': 0.11},
    'peking_duck': {'sugar': 0.66, 'carbs': 3.78, 'protein': 19.88, 'fat': 18.06},
    'pho': {'sugar': 1, 'carbs': 83, 'protein': 6, 'fat': 0.5},
    'pizza': {'sugar': 5.32, 'carbs': 23.88, 'protein': 11.98, 'fat': 11.02},
    'pork_chop': {'sugar': 0, 'carbs': 7.82, 'protein': 18.4, 'fat': 9.04},
    'poutine': {'sugar': 0.5, 'carbs': 22, 'protein': 9.4, 'fat': 19},
    'prime_rib': {'sugar': 0, 'carbs': 0, 'protein': 23.5, 'fat': 21.6},
    'pulled_pork_sandwich': {'sugar': 54, 'carbs': 81, 'protein': 34, 'fat': 7.5},
    'ramen': {'sugar': 3.45, 'carbs': 59.48, 'protein': 8.62, 'fat': 6.03},
    'ravioli': {'sugar': 0.22, 'carbs': 21.7, 'protein': 7.39, 'fat': 6.25},
    'red_velvet_cake': {'sugar': 5, 'carbs': 31.67, 'protein': 4.17, 'fat': 24.17},
    'risotto': {'sugar': 0.44, 'carbs': 14.22, 'protein': 1.78, 'fat': 4},
    'samosa': {'sugar': 0.4, 'carbs': 29.7, 'protein': 5.2, 'fat': 2.9},
    'sashimi': {'sugar': 0, 'carbs': 0, 'protein': 20.4, 'fat': 11.2},
    'scallops': {'sugar': 0.2, 'carbs': 14.1, 'protein': 11.1, 'fat': 12.5},
    'seaweed_salad': {'sugar': 6, 'carbs': 11, 'protein': 1, 'fat': 2.5},
    'shrimp_and_grits': {'sugar': 0.7, 'carbs': 21, 'protein': 23, 'fat': 20},
    'spaghetti_bolognese': {'sugar': 2.41, 'carbs': 22.41, 'protein': 5.34, 'fat': 2.76},
    'spaghetti_carbonara': {'sugar': 2.5, 'carbs': 133, 'protein': 44, 'fat': 33},
    'spring_rolls': {'sugar': 6, 'carbs': 26, 'protein': 8, 'fat': 1.8},
    'steak': {'sugar': 0, 'carbs': 0.13, 'protein': 34.82, 'fat': 4.83},
    'strawberry_shortcake': {'sugar': 25.9, 'carbs': 40.1, 'protein': 3.31, 'fat': 4.73},
    'sushi': {'sugar': 2.88, 'carbs': 25.48, 'protein': 8.47, 'fat': 2.02},
    'tacos': {'sugar': 0.34, 'carbs': 15.6, 'protein': 15.3, 'fat': 13.6},
    'takoyaki': {'sugar': 0, 'carbs': 18, 'protein': 4, 'fat': 1.8},
    'tiramisu': {'sugar': 21.33, 'carbs': 28, 'protein': 4, 'fat': 18.67},
    'tuna_tartare': {'sugar': 0.2, 'carbs': 1.9, 'protein': 7.7, 'fat': 6.8},
    'waffles': {'sugar': 10.67, 'carbs': 46.67, 'protein': 5.33, 'fat': 18.67}
      }

# 새로운 딕셔너리를 생성하여 정수 ID를 키로 사용
nutrition_mapping_2 = {name_to_id[name]: nutrients for name, nutrients in nutrition_mapping.items()}
print(nutrition_mapping_2)


{0: {'sugar': 12.5, 'carbs': 42.5, 'protein': 6.25, 'fat': 16.25}, 1: {'sugar': 7.14, 'carbs': 10.71, 'protein': 14.29, 'fat': 14.29}, 2: {'sugar': 12.8, 'carbs': 37.8, 'protein': 6.6, 'fat': 29}, 3: {'sugar': 0.7, 'carbs': 2, 'protein': 13, 'fat': 13}, 4: {'sugar': 0.08, 'carbs': 0.34, 'protein': 17.4, 'fat': 15.6}, 5: {'sugar': 7, 'carbs': 11, 'protein': 2, 'fat': 0.4}, 6: {'sugar': 31.1, 'carbs': 53.7, 'protein': 5.46, 'fat': 20.2}, 7: {'sugar': 0.05, 'carbs': 18.84, 'protein': 6.86, 'fat': 4.32}, 8: {'sugar': 15.1, 'carbs': 23.5, 'protein': 5.6, 'fat': 4.86}, 9: {'sugar': 2.11, 'carbs': 21.58, 'protein': 9.47, 'fat': 8.95}, 10: {'sugar': 3.34, 'carbs': 21.3, 'protein': 4.29, 'fat': 7.61}, 11: {'sugar': 1.6, 'carbs': 8.4, 'protein': 8.4, 'fat': 5.6}, 12: {'sugar': 13.1, 'carbs': 35.1, 'protein': 7.52, 'fat': 10.9}, 13: {'sugar': 6.83, 'carbs': 9.97, 'protein': 4.09, 'fat': 2.15}, 14: {'sugar': 22.63, 'carbs': 25.26, 'protein': 3.68, 'fat': 16.84}, 15: {'sugar': 1.34, 'carbs': 3.58, 

In [ ]:
def preprocess_image_with_nutrition(image, label):
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32) / 255.0

    label = label.numpy()  # 라벨을 numpy로 변환
    nutritional_info = nutrition_mapping_2.get(label, {'sugar': 0, 'carbs': 0, 'protein': 0, 'fat': 0})
    nutrition_tensor = tf.constant([nutritional_info['sugar'], nutritional_info['carbs'], nutritional_info['protein'], nutritional_info['fat']], dtype=tf.float32)
    return image, nutrition_tensor

def preprocess_function(image, label):
    image, nutrition_tensor = tf.py_function(preprocess_image_with_nutrition, [image, label], [tf.float32, tf.float32])
    image.set_shape((224, 224, 3))
    nutrition_tensor.set_shape((4,))
    return image, nutrition_tensor


In [ ]:
# 데이터셋 전처리 및 배치 처리
batch_size = 64
food101_dataset = food101_dataset.map(preprocess_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
food101_dataset = food101_dataset.cache().shuffle(1000).batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# 데이터셋이 비어 있지 않은지 확인
for image, label in food101_dataset.take(1):
    print(f"Image shape: {image.shape}, Label shape: {label.shape}")


Image shape: (64, 224, 224, 3), Label shape: (64, 4)


In [ ]:
# 데이터셋 분할
data_size = tf.data.experimental.cardinality(food101_dataset).numpy()
train_size = int(0.8 * data_size)
val_size = data_size - train_size

train_dataset = food101_dataset.take(train_size)
val_dataset = food101_dataset.skip(train_size).take(val_size)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam

#with strategy.scope():
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

model = Sequential([
    resnet_model,
    GlobalAveragePooling2D(),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(4, activation='linear')
])

learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)

model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, mode='min')

model.fit(train_dataset, epochs=5, validation_data=val_dataset, callbacks=[early_stopping, checkpoint])

model.save("food_nutrition_recognition.h5")



94765736/94765736 [==============================] - 1s 0us/step
Epoch 1/5
 69/947 [=>............................] - ETA: 12:33:16 - loss: 404.1281 - accuracy: 0.3766